# 13.10 转置卷积
- **目录**
  - 13.10.1 转置卷积的基本操作
  - 13.10.2 转置卷积的填充、步幅和多通道
  - 13.10.3 转置卷积与矩阵变换的联系


- 到目前为止，我们所见到的卷积神经网络层，例如卷积层（ 6.2节）和池化层（ 6.5节），通常会减少**下采样**输入图像的空间维度（高和宽）。
- 然而**如果输入和输出图像的空间维度相同，在以像素级分类的语义分割中将会很方便**。
  - 例如，输出像素所处的通道维可以保有输入像素在同一位置上的分类结果。
- 为了实现这一点，尤其是在空间维度被卷积神经网络层缩小后，我们可以使用另一种类型的卷积神经网络层，它可以增加**上采样中间层特征图**的空间维度。
- 在本节中，我们将介绍**转置卷积（transposed convolution）**，用于逆转下采样导致的空间尺寸减小。

--------------
- **说明：转置卷积的主要用途**
  - **上采样（Upsampling）**: 在图像生成或语义分割任务中，转置卷积经常用于**扩大**输入特征图的空间维度（即宽度和高度）。
  - **解卷积（Deconvolution）**: 在图像处理中，转置卷积也用于解卷积操作，以恢复原始信号。
  - **特征图合并（Feature Map Merging）**: 在多尺度特征融合中，转置卷积可以用于调整不同尺度的特征图，使其具有相同的大小，然后再进行合并。
  - **生成模型（Generative Models）**: 在生成对抗网络（GANs）或变分自编码器（VAEs）等生成模型中，转置卷积用于从较低维度的噪声或编码中生成高维度的输出（如图像）。
  - **语义分割（Semantic Segmentation）**: 转置卷积用于将压缩的特征图还原到原始输入图像的尺寸，以进行像素级别的分类。
  - **对象检测（Object Detection）**: 在某些对象检测算法中，转置卷积用于精细化预测边界框或对象的部分。
  - **图像重建（Image Reconstruction）**: 在超分辨率或图像修复任务中，转置卷积用于从低分辨率或不完整的输入重建高分辨率或完整的图像。
  - **时序数据处理（Temporal Data Handling）**: 虽然这不是图像处理，但转置卷积也可以用于一维数据（如音频信号或时间序列数据）的上采样或解卷积。
  - **三维数据处理（3D Data Handling）**: 在处理三维图像或体积数据时，三维转置卷积可用于类似的上采样或分割任务。
-------------

In [13]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l

## 13.10.1 转置卷积的基本操作

- 暂时忽略通道，从基本的转置卷积开始，设步幅为1且没有填充。
  - 假设有一个$n_h \times n_w$的输入张量和一个$k_h \times k_w$的卷积核。
  - 以步幅为1滑动卷积核窗口，每行$n_w$次，每列$n_h$次，共产生$n_h n_w$个中间结果。
  - 每个中间结果都是一个$(n_h + k_h - 1) \times (n_w + k_w - 1)$的张量，初始化为0。
  - 为了计算每个中间张量，输入张量中的每个元素都要乘以卷积核，从而使所得的$k_h \times k_w$张量替换中间张量的一部分。
  - 请注意，每个中间张量被替换部分的位置与输入张量中元素的位置相对应。
  - 最后，所有中间结果相加以获得最终结果。
- 例如， 图13.10.1解释了如何为$2\times 2$的输入张量计算卷积核为$2\times 2$的转置卷积。
<center><img src='../img/trans_conv.svg'></center>
<center>图13.10.1 卷积核为$2 \times 2$的转置卷积。阴影部分是中间张量的一部分，也是用于计算的输入和卷积核张量元素。</center>
- 我们可以对输入矩阵`X`和卷积核矩阵`K`**实现基本的转置卷积运算**`trans_conv`。


In [14]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

- 与通过卷积核“减少”输入元素的**常规卷积**（在 6.2节中）相比，转置卷积通过卷积核“广播”输入元素，从而产生**大于输入的输出**。
- 我们可以通过 图13.10.1来构建输入张量`X`和卷积核张量`K`从而**验证上述实现输出**。
- 此实现是基本的二维转置卷积运算。


In [15]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

- 或者，当输入`X`和卷积核`K`都是四维张量时，可以**使用高级API获得相同的结果**。

In [16]:
## 4个维度
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
## 转置卷积
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
## 转置卷积的权重或卷积核
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

## 13.10.2 转置卷积的填充、步幅和多通道
- 与常规卷积不同，在转置卷积中，填充被应用于卷积操作的**输出**（常规卷积将填充应用于输入）。
- 例如，当将高和宽两侧的填充数指定为1时，转置卷积的输出中将**删除**第一和最后的行与列。
- 转置卷积输出的大小计算公式：
  $$
  \text{输出尺寸} = (\text{输入尺寸} - 1) \times \text{stride} + \text{kernel\_size} - 2 \times \text{padding} \tag{13.10.1}
  $$

In [17]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

- 在转置卷积中，步幅被指定为中间结果（输出），而不是输入。
- 使用图13.10.1中相同输入和卷积核张量，将步幅从1更改为2会增加中间张量的高和宽，因此输出张量在图13.10.2中。

<center><img src='../img/trans_conv_stride2.svg'></center>
<center>图13.10.2 卷积核为$2 \times 2$，步幅为2的转置卷积。阴影部分是中间张量的一部分，也是用于计算的输入和卷积核张量元素。</center>

- 以下代码可以验证 图13.10.2中步幅为2的转置卷积的输出。

In [18]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

- 对于**多个**输入和输出通道，转置卷积与常规卷积以**相同方式**运作。
- 假设输入有$c_i$个通道，且转置卷积**为每个输入通道分配**了一个$k_h\times k_w$的卷积核张量。
  - 当指定多个输出通道时，每个输出通道将有一个$c_i\times k_h\times k_w$的卷积核。
- 同样，如果我们将$\mathsf{X}$代入卷积层$f$来输出$\mathsf{Y}=f(\mathsf{X})$，并创建一个与$f$具有相同的超参数、但输出通道数量是$\mathsf{X}$中通道数的转置卷积层$g$，那么$g(Y)$的形状将与$\mathsf{X}$相同。
- 下面的示例可以解释这一点。


In [19]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

In [20]:
tconv(conv(X)).shape

torch.Size([1, 10, 16, 16])

## 13.10.3 转置卷积与矩阵变换的联系
- 转置卷积为何以矩阵变换命名呢？
- 让我们首先看看如何使用矩阵乘法来实现卷积。
  - 在下面的示例中，我们定义了一个$3\times 3$的输入`X`和$2\times 2$卷积核`K`，然后使用`corr2d`函数计算卷积输出`Y`。


In [21]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

- 接下来，我们将卷积核`K`重写为包含大量0的稀疏权重矩阵`W`。
  - 权重矩阵的形状是（$4$，$9$），其中非0元素来自卷积核`K`。


In [22]:
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W

W = kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

- 逐行连结输入`X`，获得了一个长度为9的矢量。
- 然后，`W`的矩阵乘法和向量化的`X`给出了一个长度为4的向量。
- 重塑它之后，可以获得与上面的原始卷积操作所得相同的结果`Y`：我们刚刚使用矩阵乘法实现了卷积。


In [23]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [24]:
 torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[27., 37.],
        [57., 67.]])

- 同样，我们可以**使用矩阵乘法来实现转置卷积**。
- 在下面的示例中，我们将上面的常规卷积$2 \times 2$的输出`Y`作为转置卷积的输入。
- 想要通过矩阵相乘来实现它，我们只需要将权重矩阵`W`的形状转置为$(9, 4)$。


In [25]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [26]:
## 转置卷积与转置权重矩阵操作结果对比
Z, torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

(tensor([[ 27.,  91.,  74.],
         [138., 400., 282.],
         [171., 429., 268.]]),
 tensor([[ 27.,  91.,  74.],
         [138., 400., 282.],
         [171., 429., 268.]]))

- 抽象来看，给定输入向量$\mathbf{x}$和权重矩阵$\mathbf{W}$，卷积的前向传播函数可以通过将其输入与权重矩阵相乘并输出向量$\mathbf{y}=\mathbf{W}\mathbf{x}$来实现。
- 由于反向传播遵循链式法则和$\nabla_{\mathbf{x}}\mathbf{y}=\mathbf{W}^\top$，卷积的反向传播函数可以通过将其输入与转置的权重矩阵$\mathbf{W}^\top$相乘来实现。
- 因此，转置卷积层能够交换卷积层的正向传播函数和反向传播函数：它的正向传播和反向传播函数将输入向量分别与$\mathbf{W}^\top$和$\mathbf{W}$相乘。

## 小结

* 与通过卷积核减少输入元素的常规卷积相反，转置卷积通过卷积核广播输入元素，从而产生形状大于输入的输出。
* 如果我们将$\mathsf{X}$输入卷积层$f$来获得输出$\mathsf{Y}=f(\mathsf{X})$并创造一个与$f$有相同的超参数、但输出通道数是$\mathsf{X}$中通道数的转置卷积层$g$，那么$g(Y)$的形状将与$\mathsf{X}$相同。
* 我们可以使用矩阵乘法来实现卷积。转置卷积层能够交换卷积层的正向传播函数和反向传播函数。


---------
- **说明：权重矩阵$W$的构造过程**
  - 以输入 $X \in \mathbb{R}^{3 \times 3}$ 和卷积核 $K \in \mathbb{R}^{2 \times 2}$ 为例：
    $$
X = \begin{bmatrix}
0 & 1 & 2 \\
3 & 4 & 5 \\
6 & 7 & 8
\end{bmatrix}, \quad
K = \begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
   $$

  - 常规卷积的输出 $Y \in \mathbb{R}^{2 \times 2}$ 为：
  $$
Y = \begin{bmatrix}
0\cdot1 + 1\cdot2 + 3\cdot3 + 4\cdot4 & 1\cdot1 + 2\cdot2 + 4\cdot3 + 5\cdot4 \\
3\cdot1 + 4\cdot2 + 6\cdot3 + 7\cdot4 & 4\cdot1 + 5\cdot2 + 7\cdot3 + 8\cdot4
\end{bmatrix} = \begin{bmatrix}
27 & 37 \\
57 & 67
\end{bmatrix}
  $$

  - **步骤1：输入向量化**
  $$
\mathbf{x} = [0, 1, 2, 3, 4, 5, 6, 7, 8]^\top 
  $$
  - **步骤2：构建权重矩阵 $W$**。输出 $Y$ 有 $ 2 \times 2 = 4 $ 个元素，因此 $W$ 的形状为 $ 4 \times 9$。每行对应一个输出位置：
    - （1） **输出 $Y_{00}$**：卷积核覆盖 $X$ 的左上角区域（位置 $(0,0)$）：

       $$
   \text{覆盖输入索引：} (0,0), (0,1), (1,0), (1,1) \quad \Rightarrow \quad \mathbf{x} \text{ 的索引：} 0, 1, 3, 4
       $$

       $$
   W_{0,:} = [\textcolor{red}{1}, \textcolor{red}{2}, 0, \textcolor{red}{3}, \textcolor{red}{4}, 0, 0, 0, 0]
       $$
      
    - （2）**输出 $Y_{01}$**：卷积核右移一步（位置 $(0,1)$）：
       $$
   \text{覆盖输入索引：} (0,1), (0,2), (1,1), (1,2) \quad \Rightarrow \quad \mathbf{x} \text{ 的索引：} 1, 2, 4, 5
       $$

       $$
   W_{1,:} = [0, \textcolor{red}{1}, \textcolor{red}{2}, 0, \textcolor{red}{3}, \textcolor{red}{4}, 0, 0, 0]
       $$
  
     - （3）**输出 $Y_{10}$**：卷积核下移一步（位置 $(1,0)$）：

       $$
   \text{覆盖输入索引：} (1,0), (1,1), (2,0), (2,1) \quad \Rightarrow \quad \mathbf{x} \text{ 的索引：} 3, 4, 6, 7
       $$

      $$
   W_{2,:} = [0, 0, 0, \textcolor{red}{1}, \textcolor{red}{2}, 0, \textcolor{red}{3}, \textcolor{red}{4}, 0]
      $$
    - （4）**输出 $Y_{11}$**：卷积核右下角（位置 $(1,1)$）：

       $$
       \text{覆盖输入索引：} (1,1), (1,2), (2,1), (2,2) \quad \Rightarrow \quad \mathbf{x} \text{ 的索引：} 4, 5, 7, 8
      $$

      $$
   W_{3,:} = [0, 0, 0, 0, \textcolor{red}{1}, \textcolor{red}{2}, 0, \textcolor{red}{3}, \textcolor{red}{4}]
      $$

      最终，权重矩阵 $W$ 为：
      $$
W = \begin{bmatrix}
1 & 2 & 0 & 3 & 4 & 0 & 0 & 0 & 0 \\
0 & 1 & 2 & 0 & 3 & 4 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 2 & 0 & 3 & 4 & 0 \\
0 & 0 & 0 & 0 & 1 & 2 & 0 & 3 & 4
\end{bmatrix}
      $$

  - **步骤3：矩阵乘法验证**
    $$
\mathbf{y} = W \mathbf{x} = \begin{bmatrix}
1\cdot0 + 2\cdot1 + 3\cdot3 + 4\cdot4 \\
1\cdot1 + 2\cdot2 + 3\cdot4 + 4\cdot5 \\
1\cdot3 + 2\cdot4 + 3\cdot6 + 4\cdot7 \\
1\cdot4 + 2\cdot5 + 3\cdot7 + 4\cdot8
\end{bmatrix} = \begin{bmatrix}
27 \\
37 \\
57 \\
67
\end{bmatrix}
    $$

    将 $\mathbf{y}$ 变形为 $2 \times 2$ 即可得到 $Y$。

-----